# Worldwide Earthquake Events API - Silver Layer Processing

In [5]:
from datetime import date, timedelta

start_date = date.today() - timedelta(7)


StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, 8, Finished, Available, Finished)

In [7]:
# df now is a Spark DataFrame containing JSON data
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")

StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, 9, Finished, Available, Finished)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: abfss://1aa7eacd-5707-4511-a838-d5af31b4ac92@onelake.dfs.fabric.microsoft.com/4662a9e8-5adc-4cac-94af-0a9a01d09675/Files/2026-01-08_earthquake_data.json.

In [ ]:
# Reshape earthquake data by extracting and renaming key attributes for further analysis.
df = \
df.\
    select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )

StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, -1, Cancelled, , Cancelled)

In [ ]:
# Convert 'time' and 'updated' columns from milliseconds to timestamp format for clearer datetime representation.
df = df.\
    withColumn('time', col('time')/1000).\
    withColumn('updated', col('updated')/1000).\
    withColumn('time', col('time').cast(TimestampType())).\
    withColumn('updated', col('updated').cast(TimestampType()))

StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, -1, Cancelled, , Cancelled)

In [ ]:
# appending the data to the gold table
df.write.mode('append').saveAsTable('earthquake_events_silver')

StatementMeta(, ae3abdb4-4ef6-40ec-ba22-c4588f01c770, -1, Cancelled, , Cancelled)